# FATES_INCLINE_dataprep_surfacedata

Heavily inspired from NorESM-LSP notebooks and https://github.com/huitang-earth/MossLichen_testbed/blob/main/scripts/SeedClim_surfacedata_modification.ipynb 

Script by Eva Lieungh, Elin CR Aas, Hui Tang
Started 2022-11-15

### Data
Observational data from the Vestland Climate Grid  stored on OSF: 
https://osf.io/npfa9

Data files:
- https://osf.io/s9k7c, VCG_clean_gridded_daily_climate_2008-2022.csv
- VCG_clean_soil_chemistry_2009_2010_2013_2015.csv
- VCG_clean_soil_structure_2013_2014_2018.csv
- VCG_clean_soilmoisture_plotlevel_2015-2018.csv

Surface data files were downloaded from Land Sites Platform version 1 default data, subset from global files. https://ns2806k.webs.sigma2.no/EMERALD/EMERALD_platform/inputdata_noresm_landsites/v1.0.0/default/ 

Data files stored per site as
`inputdata/lnd/clm2/surfdata_map/{siteID}/surfdata_{siteID}_simyr2000.nc`, extracted to site-specific folders.

### This script

... loads necessary libraries and data, explores them a bit, then runs a loop to modify the surface data. 

In [1]:
# import libraries
import xarray as xr  # NetCDF data handling
import netCDF4 
import matplotlib.pyplot as plt  # Plotting
import time  # Keeping track of runtime
import json  # For reading data dictionaries stored in json format
import pandas as pd  # Tabular data analysis
import datetime as dt  # For workaround with long simulations (beyond year 2262)
import statistics as stats # For mean and other calculations
from pathlib import Path  # For easy path handling

In [36]:
# define LSP site identities and corresponding names 
siteID = ["ALP1","ALP2","ALP3","ALP4","SUB1","SUB2","SUB3","SUB4","BOR1","BOR2","BOR3","BOR4"]
siteID1 = ["Ulvehaugen","Lavisdalen","Gudmedalen","Skjelingahaugen",
           "Alrust","Hogsete","Rambera","Veskre",
           "Fauske","Vikesland","Arhelleren","Ovstedalen"] 
siteID2 = ["Ulvehaugen","Lavisdalen","Gudmedalen","Skjellingahaugen",
           "Alrust","Hogsete","Rambera","Veskre",
           "Fauske","Vikesland","Arhelleren","Ovstedalen"]
siteID3 = ["ULV","LAV","GUD","SKJ","ALR","HOG","RAM","VES","FAU","VIK","ARH","OVS"]

In [48]:

# set path observational data
observations_path = Path(f"C:/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/3_FATES_INCLINE/data/OSF_VCG")

# set path for where to store modified data
modified_surfacedata_path = str(Path(f"C:/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/3_FATES_INCLINE/data_processed/surfacedata"))

Create surface data files to be modified by copying the default ones into the modified_surfacedata_path

In [ ]:
#%%bash
#cp -a ../data/LSPv1_default_inputfiles/. ../data_processed/surfacedata/

Read in the observations from files, print available variables and other relevant info

In [4]:
soil_moisture = pd.read_csv(observations_path / "VCG_clean_soilmoisture_plotlevel_2015-2018.csv",
                            index_col=None)
soil_moisture.head()

,year,date,siteID,turfID,blockID,blockID_FC,replicate,value,weather,recorder,comments,transcriber_comment
0,2015.0,2015-06-03,Alrust,31TTC,Alr2,2,1,28.7,cloudy,siri,NaN,NaN
1,2015.0,2015-06-03,Alrust,31TTC,Alr2,2,2,30.8,cloudy,siri,NaN,NaN
2,2015.0,2015-06-03,Alrust,31TTC,Alr2,2,3,35.3,cloudy,siri,NaN,NaN
3,2015.0,2015-06-03,Alrust,31TTC,Alr2,2,4,26.7,cloudy,siri,NaN,NaN
4,2015.0,2015-06-03,Alrust,7TT233,Alr2,2,1,34.5,cloudy,siri,NaN,NaN


In [56]:
soil_chemistry = pd.read_csv(observations_path / "VCG_clean_soil_chemistry_2009_2010_2013_2015.csv",
                             index_col=None)
print(soil_chemistry.variable.unique())
print(soil_chemistry.siteID_dest.unique()) # matches siteID1 list

['pH' 'loi' 'NO3N' 'NH4N' 'available_N' 'soil_depth' 'N_content'
 'C_content' 'CN_ratio']
['Arhelleren' 'Fauske' 'Gudmedalen' 'Hogsete' 'Lavisdalen' 'Rambera'
 'Skjelingahaugen' 'Ulvehaugen' 'Veskre' 'Vikesland' 'Ovstedalen' 'Alrust']


In [59]:
soil_structure = pd.read_csv(observations_path / "VCG_clean_soil_structure_2013_2014_2018.csv", 
                             index_col=None)
print(soil_structure.variable.unique())
print(soil_structure.siteID.unique()) # matches siteID1

# for some reason, only the alpine sites have data for clay, silt and sand 
print(soil_structure[soil_structure["variable"] == "sand"].siteID.unique())

['soil_depth' 'clay' 'silt' 'sand' 'bulk_density']
['Arhelleren' 'Alrust' 'Fauske' 'Gudmedalen' 'Hogsete' 'Lavisdalen'
 'Ovstedalen' 'Rambera' 'Skjelingahaugen' 'Ulvehaugen' 'Veskre'
 'Vikesland' 'ARH']
['Ulvehaugen' 'Lavisdalen' 'Gudmedalen' 'Skjelingahaugen']


In [34]:
INCLINE_metadata = pd.read_csv("C:/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/3_FATES_INCLINE/data/INCLINE/INCLINE_metadata.csv", sep=";", index_col=None)
INCLINE_metadata.head()

,turfID,plotID,siteID,blockID,plot,OTC,treatment,precipitation_1960-1990,precipitation_2009-2019,temperature_1960-1990,temperature_2009-2019,coordinate_N,coordinate_E,elevation,slope,aspect
0,Gud_1_1,Gud_1_1,Gudmedalen,1,1,W,R,1925,2130,"5,87","6,04",6049880,710301,1117,16,S/SW
1,Gud_1_2,Gud_1_2,Gudmedalen,1,2,W,N,1925,2130,"5,87","6,04",6049880,710301,1117,17,S/SW
2,Gud_1_3,Gud_1_3,Gudmedalen,1,3,W,C,1925,2130,"5,87","6,04",6049880,710301,1117,29,S/SW
3,Gud_1_4,Gud_1_4,Gudmedalen,1,4,C,E,1925,2130,"5,87","6,04",6049879,710301,1117,17,S/SW
4,Gud_1_5,Gud_1_5,Gudmedalen,1,5,C,C,1925,2130,"5,87","6,04",6049879,710302,1117,14,S/SW


### In one big loop, for each site, 

1. subset the variables from observation data sets
2. calculate means of the variables,
3. replace the default in the surfacedata file, 
4. and save the file in the modified_surfacedata_path

Subset carbon content, calculate site mean. Needs to be combined with bulk density to convert to kg/m3

From CLM tech note about soil:
"The soil texture and organic matter content determine soil thermal and hydrologic properties (sections 6.3 and 7.4.1). The International Geosphere-Biosphere Programme (IGBP) soil dataset (Global Soil Data Task 2000) of 4931 soil mapping units and their sand and clay content for each soil layer were used to create a mineral soil texture dataset (Bonan et al. 2002b). Soil organic matter data is merged from two sources. The majority of the globe is from ISRIC-WISE (Batjes, 2006). The high latitudes come from the 0.25o version of the Northern Circumpolar Soil Carbon Database (Hugelius et al. 2012). Both datasets report carbon down to 1m depth. Carbon is partitioned across the top seven CLM4 layers (\sim1m depth) as in Lawrence and Slater (2008)."
https://escomp.github.io/ctsm-docs/versions/release-clm5.0/html/tech_note/Ecosystem/CLM50_Tech_Note_Ecosystem.html#surface-characterization

#### more variables to be added to the loop below:

plant_cover_obs # regn ut bare ground vs plantedekke fra smårute-cover av bare+rock fra INCLINE community fil / rock+bare på plottnivå fra SeedClim


In [54]:
for i in range(0,4):
    print("-------------------------------------")
    print("site: ", siteID[i], siteID2[i])

    #------------- GET OBSERVATION DATA -------------# 

    # Subset variables, calculate means, do unit conversions

    # Organic matter #
    carbon_observed = soil_chemistry[(soil_chemistry["siteID_dest"]==siteID1[i])
                                     & (soil_chemistry["variable"] == "C_content")]
    soil_bulk_density = soil_structure[(soil_structure["siteID"]==siteID1[i]) 
                                       & (soil_structure["variable"] == "bulk_density")]
    carbon_mean = stats.mean(carbon_observed["value"])
    soil_bulk_density_mean = stats.mean(soil_bulk_density["value"])
    print("mean carbon content (%): ", carbon_mean)
    print("bulk density (g/cm^3):   ", soil_bulk_density_mean)
        # observed bulk density: g/cm3, observed carbon content: %, model needs organic: kg/m3. 
        # To get total organic matter (not just C), divide by 0.58   
    org_obs = (soil_bulk_density_mean*1000)*(carbon_mean/100)/0.58 
    org_obs = min(110, org_obs)      # model assumes less than 130 kg/m3 organic matter 
    print("organic matter for model (kg/m3): ", org_obs)

    # percent sand # (only for ALP1-4)
    sand_observed = soil_structure[(soil_structure["siteID"]==siteID1[i]) 
                                       & (soil_structure["variable"] == "sand")]
    sand_obs_mean = stats.mean(sand_observed["value"]) * 100
    print("sand %: ", sand_obs_mean)

    ## silt #
    #silt_observed = soil_structure[(soil_structure["siteID"]==siteID1[i]) 
    #                                   & (soil_structure["variable"] == "silt")]
    #silt_obs_mean = stats.mean(silt_observed["value"]) * 100
    #print("silt %: ", silt_obs_mean)

    # clay #
    clay_observed = soil_structure[(soil_structure["siteID"]==siteID1[i]) 
                                       & (soil_structure["variable"] == "clay")]
    clay_obs_mean = stats.mean(clay_observed["value"]) * 100
    print("clay %: ", clay_obs_mean)

    # Soil depth#
    soil_depth = soil_structure[(soil_structure["siteID"]==siteID1[i]) 
                                       & (soil_structure["variable"] == "soil_depth")]
    soil_depth_mean = stats.mean(soil_depth["value"]) / 100 # convert cm to m
    print("soil depth, m: ", soil_depth_mean)

    # Slope # (NB! from INCLINE data - not identical to SeedClim and FunCaB data)
    slope_observed = INCLINE_metadata[INCLINE_metadata["siteID"]==siteID2[i]]
    slope_obs_mean = stats.mean(slope_observed["slope"])
    print("slope: ",slope_obs_mean)

    # plant and bare ground cover # - set to a fixed number for now! To be calculated from data
    plant_cover_obs = 90 # % cover

    # plant height # 
    plant_height_obs = 15/100
    #MONTHLY_HEIGHT_TOP

    # grazing??? #
    # CONST_GRAZING (longname grazing of herbaceous pfts)

    #------------- MODIFY SURFACE DATA VARIABLES -------------#

    # open site-specific default surface data file to be modified
    dset = netCDF4.Dataset(modified_surfacedata_path + "/" + siteID[i] + "/surfdata_" + siteID[i] + "_simyr2000.nc", 
                           'r+')
    
    # modify cover of specific PFTs
    dset['PCT_NAT_PFT'][0,:,:] = 100-plant_cover_obs # index 0: barren ground

    # modify land cover fractions (set whole gridcell to natural vegetation)
    dset['PCT_NATVEG'][:,:] = 100
    dset['PCT_CROP'][:,:] = 0
    dset['PCT_CFT'][:,:,:] = 0
    dset['PCT_WETLAND'][:,:] = 0
    dset['PCT_LAKE'][:,:] = 0
    dset['PCT_GLACIER'][:,:] = 0
    dset['PCT_URBAN'][:,:,:] = 0

    # Modify soil properties
    dset['ORGANIC'][0:3,:,:] = org_obs        # the layers of soil to modify depending on the availability of the data
    dset['PCT_SAND'][:,:,:] = sand_obs_mean
    dset['PCT_CLAY'][:,:,:] = clay_obs_mean
    dset['zbedrock'][:,:] = soil_depth_mean

    # Modify topography
    dset['SLOPE'][:,:] = slope_obs_mean

    dset.close()

-------------------------------------
site:  ALP1 Ulvehaugen
mean carbon content (%):  9.084
bulk density (g/cm^3):    0.2949073635697664
organic matter for model (kg/m3):  46.18859466668548
sand %:  19.72681304988417
clay %:  34.11610445379923
soil depth, m:  0.072865
slope:  14.45
-------------------------------------
site:  ALP2 Lavisdalen
mean carbon content (%):  4.7925
bulk density (g/cm^3):    0.358264386942836
organic matter for model (kg/m3):  29.603139214199
sand %:  17.31825456291833
clay %:  32.26604699983674
soil depth, m:  0.09205999999999999
slope:  24.4
-------------------------------------
site:  ALP3 Gudmedalen
mean carbon content (%):  7.558125
bulk density (g/cm^3):    0.43329595886157235
organic matter for model (kg/m3):  56.46387963914865
sand %:  7.1183379993004
clay %:  47.285209745810704
soil depth, m:  0.08664999999999999
slope:  17.325
-------------------------------------
site:  ALP4 Skjellingahaugen
mean carbon content (%):  2.1644
bulk density (g/cm^3):   